In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
import mlpp.pp.hawkes as hk
from mlpp.base.utils import TimeFunction
from nphc.metrics import rel_err, rank_corr

# Definition of $||\Phi||$

In [23]:
d = 100
mu = 0.05 * np.ones(d)
blocks = [d/2,d/2]
L = []
L.append(np.ones((blocks[0],blocks[0])))
L.append(2*np.ones((blocks[1],blocks[1])))
Alpha_true = scipy.linalg.block_diag(*L)
# add noise
#Alpha_true += 0.01
Alpha_true[10,17] += 3.
Alpha_true = .5*(Alpha_true+Alpha_true.T)
Alpha_true /= 120

### Check if $||\Phi|| = Alpha_{true}$, can generate a stable Hawkes process

In [24]:
_, s, _ = np.linalg.svd(Alpha_true)
print(s.max())
assert s.max() < 1, "Alpha_true cannot generate a stable Hawkes process"

0.833333333333


## Major key for exp kernel: 
### Since hMax=40 encodes the support, we ensure \beta is not too small for estimation
Criterion on exp kernel gives us $\beta_{\min}$ such that $$\exp(-\mbox{hMax} \times \beta_\min) = 10^{-3}$$

In [25]:
hMax = 40
from math import log
beta_min = log(1000) / hMax
print("We have")
print("    beta_min = ",beta_min)

We have
    beta_min =  0.17269388197455343


## Fill matrix of $\beta$

In [27]:
LL = []
LL.append(100*beta_min*np.ones((blocks[0],blocks[0])))
LL.append(beta_min*np.ones((blocks[1],blocks[1])))
Beta = scipy.linalg.block_diag(*LL)
Beta[10,17] = 2*beta_min
Beta = .5*(Beta + Beta.T)

## Simulation of the process

In [ ]:
kernels = [[hk.HawkesKernelExp(a, b) for (a, b) in zip(a_list, b_list)] for (a_list, b_list) in zip(Alpha_true, Beta)]
h = hk.Hawkes(kernels=kernels, mus=list(mu))
T_max = 100000
h.simulate(T_max)

## Use the class Cumulants

In [ ]:
from nphc.cumulants import Cumulants
N = h.get_full_process()
cumul = Cumulants(N,hMax=40)
# Compute useful integrated cumulants
cumul.compute_all()

# Estimation procedure

In [ ]:
from nphc.nphc2.solver import admm
from nphc.prox import *
X0 = np.eye(d)
#X0 = np.ones(d**2).reshape(d,d)
rho = .01
maxiter = 10000

In [ ]:
res = []
for prox in [prox_sq_frob, prox_l1, prox_nuclear]:
    X_ = admm(cumul, prox, X0, X0, Alpha_true, rho=rho, maxiter=maxiter)
    res.append(X_)

In [ ]:
X_frob = res[0]
X_l1 = res[1]
X_nuc = res[2]

# Results

In [ ]:
import pandas as pd
cols = ['RelErr', 'RankCorr']
index = ['frobenius', '\ell_1', 'nuclear norm']
results = np.zeros((len(index),len(cols)))
for i, X in enumerate([X_frob, X_l1, X_nuc]):
    results[i][0] = rel_err(Alpha_true, X)
    results[i][1] = rank_corr(Alpha_true, X)
df = pd.DataFrame(results, index=index, columns=cols)
df

# Heatmaps

In [ ]:
plt.pcolor(Alpha_true,cmap=plt.cm.Blues)
plt.title("Alpha_true")
plt.colorbar()
plt.show()
plt.close()

In [ ]:
eps = 1e-10
plt.pcolor(np.log(eps+Beta),cmap=plt.cm.Blues)
plt.title("Beta")
plt.colorbar()
plt.show()
plt.close()

In [ ]:
from nphc.cumulants import get_C
cumul.set_C()
plt.pcolor(cumul.C,cmap=plt.cm.Reds)
plt.title("Covariance matrix")
plt.colorbar()
plt.show()
plt.close()

In [ ]:
for i,X in enumerate(res):
    plt.pcolor(X,cmap=plt.cm.Blues)
    if i == 0:
        title = 'X_frob'
    elif i == 1:
        title = 'X_l1'
    else:
        title = 'X_nuc'
    plt.title(title)
    plt.colorbar()
    plt.show()
    plt.close()